In [ ]:
import pandas as pd
import numpy as np
import requests

In [ ]:
from dateutil.parser import parse

In [ ]:
import datetime

In [ ]:
from thredds_crawler.crawl import Crawl

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
chrome_options = Options()  
chrome_options.add_argument("--headless")  

# SOEST catalog

In [ ]:
driver = webdriver.Chrome('/Users/brey/Downloads/chromedriver',options=chrome_options)
driver.get('https://uhslc.soest.hawaii.edu/data/?fd')

In [ ]:
table = driver.find_element_by_tag_name('body')

In [ ]:
table_html = table.get_attribute('innerHTML')
#table_html

In [ ]:
uh = pd.read_html(table_html)[0]

In [ ]:
uh.head()

In [ ]:
uh = uh.loc[:,['UH#','GLOSS#','Location','Country','Latitude','Longitude','Start','End']]

In [ ]:
uh[uh['Location'].str.contains("Majuro")]

In [ ]:
uh[uh['Country'].str.contains("Hong Kong")]

### extract links

    - NOTE : This used to work but not anymore

In [ ]:
crawler = Crawl('http://uhslc.soest.hawaii.edu/thredds/uhslc_fast_hourly.html', debug=True)

In [ ]:
info = pd.DataFrame([(d.id, d.name, s.get("url")) for d in crawler.datasets for s in d.services if s.get("service") == "OPENDAP"])

In [ ]:
info.columns = ['id','name','link']

In [ ]:
info.head()

In [ ]:
#select a location
loc = info[info['name'].str.contains("Quarry")]
loc

In [ ]:
#get the link
loc['link']

In [ ]:
tg_link = loc['link'].values[0]

In [ ]:
tg_link

### Add links to the Dataframe

In [ ]:
info.head()

In [ ]:
#create a column with only the ID 
info['UH#'] = info.loc[:,'id'].apply(lambda row: row[3:6])

In [ ]:
info.head()

In [ ]:
info['UH#'] = info['UH#'].apply(pd.to_numeric)

In [ ]:
uh = uh.merge(info, left_on=['UH#'], right_on=['UH#'])

In [ ]:
uh.head()

In [ ]:
uh[uh['Location'].str.contains("Majuro")]

In [ ]:
uh[uh['Location'].str.contains("Majuro")]['link'].values[0]

In [ ]:
#save DataFrame
uh.to_csv('soest.csv',encoding='utf-8',index=False)

### with erddappy

#####  	UHSLC Fast Delivery Tide Gauge Data (hourly)

In [ ]:
from erddapy import ERDDAP


e = ERDDAP(
  server="https://uhslc.soest.hawaii.edu/erddap",
  protocol="tabledap",
)

e.response = "csv"
e.dataset_id = "global_hourly_fast"
e.constraints = {
}
e.variables = [
    "station_name",
    "station_country",
    "station_country_code",
    "uhslc_id",
    "gloss_id",
    "ssc_id",
    "last_rq_date",
]

df = e.to_pandas()

In [ ]:
df

##### JASL/UHSLC Research Quality Tide Gauge Data (hourly) 

In [ ]:
from erddapy import ERDDAP


e = ERDDAP(
  server="https://uhslc.soest.hawaii.edu/erddap",
  protocol="tabledap",
)

e.response = "csv"
e.dataset_id = "global_hourly_rqds"
e.constraints = {
}
e.variables = [
    "station_name",
    "station_country",
    "station_country_code",
    "uhslc_id",
    "gloss_id",
    "ssc_id",
    "decimation_method",
    "reference_code",
    "reference_offset",
    
]

df = e.to_pandas()

In [ ]:
df

### get data

In [ ]:
soest = pd.read_csv('soest.csv')

In [ ]:
soest.head()

In [ ]:
soest[soest['Location'].str.contains("Majuro")]

In [ ]:
tg_link=soest[soest['Location'].str.contains("Majuro")]['link'].values[0]

In [ ]:
dataset = open_url(tg_link)

In [ ]:
dataset.attributes

In [ ]:
dataset.keys() # available variables

In [ ]:
time = dataset['time']

In [ ]:
time.attributes

In [ ]:
t = dataset['time'][:]

In [ ]:
t

### transform time to readable format

In [ ]:
time_ref = time.attributes['units'].split()[-1]

In [ ]:
tref = parse(time_ref)

In [ ]:
tref

In [ ]:
tm = [tref + datetime.timedelta(days=d) for d in time]

In [ ]:
tm[0], tm[-1]


### get the index of date range

In [ ]:
index_start = bisect.bisect_right(tm, datetime.datetime(2016,8,20).replace(tzinfo=tm[0].tzinfo)) # note the conversion to the same timezone aware  value

In [ ]:
index_end = bisect.bisect_right(tm, datetime.datetime(2016,9,1).replace(tzinfo=tm[0].tzinfo))

In [ ]:
index_start, index_end

In [ ]:
rt = np.array(tm)[index_start:index_end]

In [ ]:
rt.shape

In [ ]:
dataset['sea_surface_height_above_reference_level'].attributes

In [ ]:
h = dataset['sea_surface_height_above_reference_level']#[index_start:index_end].array

In [ ]:
h

In [ ]:
lat = dataset['latitude'][:]
lon = dataset['longitude'][:]

In [ ]:
h.shape

In [ ]:
hh = h.data.reshape(-1)

In [ ]:
q = pd.DataFrame({'time':rt,'tide_gauge':hh})
q = q.set_index('time')

In [ ]:
#convert to m
q.tide_gauge = q.tide_gauge/1000.

In [ ]:
q.head()

In [ ]:
ax = q.plot()